In [24]:
# external dependencies
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc
import plotly.express as px

In [25]:
# constants file
from constants import *
from graphs import *


### Population Dataset

In [26]:
# load dataset for US population
df_pop = pd.read_csv('US_Population.csv')
df_pop.dropna()
df_pop.head()

,Year,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2016,4860545,741522,6941072,2989918,39250017,5540545,3576452,952065,20612439,...,865454,6651194,27862596,3051217,624594,8411808,7288000,1831102,5778708,585501
1,2017,4874747,739786,7044008,3001345,39536653,5607154,3573880,961939,20928863,...,869666,6715984,28304596,3101833,623657,8470020,7423362,1818157,5790186,584910
2,2018,4887681,735139,7158024,3009733,39776830,5691287,3573297,971180,21244317,...,878698,6771631,28704330,3153550,624344,8517685,7535591,1804291,5807406,578668
3,2019,4903185,731158,7278717,3017804,39576757,5758736,3571520,981822,21538187,...,882235,6829174,29145505,3205958,624358,8565256,7614893,1792147,5822434,577601
4,2020,4921532,727890,7421401,3029887,39368078,5845526,3565287,990837,21899341,...,886667,6886834,29618533,3251617,623989,8616207,7693612,1778070,5837466,567025


### Accident Dataset

In [27]:

# import dataset for US accident
df_acc = pd.read_csv('US_Accidents.csv')

# remove unecessary columns
df_acc.drop(['Source', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description', 'Airport_Code'], axis=1, inplace=True)

# remove null values
df_acc.dropna()

# sample a subset of the database (for efficency)
df_acc = df_acc.sample(n = 70000, random_state = 351)

# Convert 'Start_Time' and 'End_Time to datetime
df_acc['Start_Time'] = pd.to_datetime(df_acc['Start_Time'],format='ISO8601')
df_acc['End_Time'] = pd.to_datetime(df_acc['End_Time'],format='ISO8601')

# remove year 2023 because it is incomplete
df_acc = df_acc[(df_acc['Start_Time'].dt.year != 2023)]

df_acc.head()

,ID,Severity,Start_Time,End_Time,Distance(mi),Street,City,County,State,Zipcode,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
2909380,A-2919259,3,2018-02-28 19:10:31,2018-02-28 19:55:17,0.00,Carson St,Lakewood,Los Angeles,CA,90715,...,False,False,False,False,False,False,Night,Night,Night,Day
4123847,A-4154878,2,2022-11-22 16:09:03,2022-11-22 17:26:55,0.99,I-40 W,Nashville,Davidson,TN,37205,...,False,False,False,False,False,False,Day,Day,Day,Day
7006131,A-7055452,2,2020-06-24 20:28:00,2020-06-24 21:02:05,0.00,I-94 E,Minneapolis,Hennepin,MN,55412,...,False,False,False,False,False,False,Day,Day,Day,Day
1191012,A-1200791,2,2021-01-08 18:01:03,2021-01-08 18:47:15,0.00,S Choctaw Dr,Baton Rouge,East Baton Rouge,LA,70815-8901,...,False,False,False,False,True,False,Night,Night,Day,Day
453540,A-453553,2,2017-05-11 08:05:29,2017-05-11 08:34:56,0.02,Meridian Ave N,Seattle,King,WA,98133-8540,...,False,False,False,False,True,False,Day,Day,Day,Day


__Dashboard Layout__

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([

    # title
    dbc.Row(dbc.Col(html.H1("USA Accidents"), width="auto"), justify="center",), 
    html.Br(),

    # top buttons 
    dbc.Row([

        # time interval select
        dbc.Col([
            dbc.Row(
                dbc.Col(
                    html.P("Time Interval: "), width="auto"
                ), 
                justify="center"
            ),
            dbc.Row(
                dbc.Select(
                    options=[
                        {"label":"Monthly", "value":"Monthly"}, 
                        {"label":"Yearly", "value":"Yearly"}, 
                        {"label":"Daily", "value":"Daily"},
                        {"label":"Hourly", "value":"Hourly"}  
                    ],
                    value='Monthly',
                    id="Time-Interval-Select"
                )
            )
        ]),

        # view mode select
        dbc.Col([
            dbc.Row(
                dbc.Col(
                    html.P("View Mode: "), width="auto"
                ), 
                justify="center"
            ),
            dbc.Row(
                dbc.RadioItems(
                    options=[
                        {"label":"Grouped", "value":"Grouped"}, 
                        {"label":"Separated", "value":"Separated"}
                    ],
                    value = 'Grouped',
                    id='ViewMode-Select'
                )
            )
        ])
    ]),

    # first graph row
    dbc.Row([

        # top-left graph
        dbc.Col([
            # barchart
            dcc.Graph(id='Temporal-Distribution-acc-graph')
        ])

    ])

])

app.run(debug=True)

In [28]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([

    # title row
    dbc.Row(dbc.Col(html.H1("USA Accidents"), width="auto"), justify="center",), 
    html.Br(),

    # top graphs row
    dbc.Row([

        # top-left graph col
        dbc.Col([
            # top-left graph options
            dbc.Row([
                # time interval select
                dbc.Col([
                    dbc.Row(
                        dbc.Col(
                            html.P("Time Interval: "), width="auto"
                        ), 
                        justify="center"
                    ),
                    dbc.Row(
                        dbc.Select(
                            options=[
                                {"label":"Monthly", "value":"Monthly"}, 
                                {"label":"Yearly", "value":"Yearly"}, 
                                {"label":"Daily", "value":"Daily"},
                                {"label":"Hourly", "value":"Hourly"}  
                            ],
                            value='Monthly',
                            id="Time-Interval-Select"
                        )
                    )
                ]),
                # view mode select
                dbc.Col([
                    dbc.Row(
                        dbc.Col(
                            html.P("View Mode: "), width="auto"
                        ), 
                        justify="center"
                    ),
                    dbc.Row(
                        dbc.RadioItems(
                            options=[
                                {"label":"Grouped", "value":"Grouped"}, 
                                {"label":"Separated", "value":"Separated"}
                            ],
                            value = 'Grouped',
                            id='ViewMode-Select'
                        )
                    )
                ])
            ]),
            # top-left graph
            dbc.Row([
                 # top-left graph
                dbc.Col([
                    # barchart
                    dcc.Graph(id='Temporal-Distribution-acc-graph')
                ])
            ])
        ]),

        # top-right graph col
        dbc.Col([
            # top-right graph option
            dbc.Row([
                dbc.Select(
                    options=[
                        {"label":"All", "value":"all"}, 
                        {"label":"2016", "value":"2016"}, 
                        {"label":"2017", "value":"2017"},
                        {"label":"2018", "value":"2018"},
                        {"label":"2019", "value":"2019"},
                        {"label":"2020", "value":"2020"},
                        {"label":"2021", "value":"2021"},
                        {"label":"2022", "value":"2022"}        
                    ],
                    value='all',
                    id="Time-Interval-Select-pie"
                )
            ]),
            dbc.Row([
                dcc.Graph(id='Severity-distribution-pie')
            ])
        ]),
    ])

])

    

app.run(debug=True)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[30], line 3, in SDPiechart(time_interval='all')
      1 @app.callback(Output("Severity-distribution-pie", "figure"),[Input("Time-Interval-Select-pie", "value")])
      2 def SDPiechart(time_interval):
----> 3     fig = PieChart(df_acc, time_interval)
        df_acc =                 ID  Severity          Start_Time            End_Time  \
2909380  A-2919259         3 2018-02-28 19:10:31 2018-02-28 19:55:17   
4123847  A-4154878         2 2022-11-22 16:09:03 2022-11-22 17:26:55   
7006131  A-7055452         2 2020-06-24 20:28:00 2020-06-24 21:02:05   
1191012  A-1200791         2 2021-01-08 18:01:03 2021-01-08 18:47:15   
453540    A-453553         2 2017-05-11 08:05:29 2017-05-11 08:34:56   
...            ...       ...                 ...                 ...   
3000186  A-3010065         2 2018-02-13 11:33:30 2018-02-13 12:18:16

__Top-left graph callback__

In [29]:
@app.callback(Output("Temporal-Distribution-acc-graph", "figure"),[Input("Time-Interval-Select", "value"), Input("ViewMode-Select", "value")])
def TDABarchart(time_interval, ViewMode):
    if(ViewMode == 'Grouped'):
        fig = SingleBarChart(df_acc, time_interval)
        return fig
    else:
        if(ViewMode == 'Separated'):
            fig = MultiBarChart(df_acc, time_interval)
            return fig
        else:
            return 0

__Top-right graph callback__

In [30]:
@app.callback(Output("Severity-distribution-pie", "figure"),[Input("Time-Interval-Select-pie", "value")])
def SDPiechart(time_interval):
    fig = PieChart(df_acc, time_interval)
    return fig

http://127.0.0.1:8050